In [1]:
import os 
import glob
from pathlib import Path
import pandas as pd 
import numpy as np 
import ast
import json 
import nibabel as nib
import imageio
import matplotlib.pyplot as plt
%matplotlib inline

/data/svcf/software/anaconda3/envs/fastai/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


# Creating all of our test images + one csv file to keep track of them all 

## Define the functionality of our pipeline: 

In [2]:
## NORMALIZATION:
def zscore_normalize(img, brainmask): 
    masked_image_values = img[brainmask==1]
    img_norm = (img-masked_image_values.mean())/np.std(masked_image_values)
    img_norm = img_norm+abs(img_norm.min())
    img_norm_masked = img_norm*brainmask
    return img_norm_masked

In [3]:
## Finding the slices in each direction with greatest amount of ROI lesion: 
def find_slices_with_max_area_roi(roi_img):
    if len(np.unique(roi_img)) >2: 
        print("ERROR! Weird ROI w/ more than 0/1 values")
    sum_roi_axial = roi_img.sum(axis = 1).sum(axis = 1)
    sum_roi_sagittal = roi_img.sum(axis= 0).sum(axis = 0)
    sum_roi_coronal = roi_img.sum(axis = 0).sum(axis = 1)
    return {'axial': int(sum_roi_axial.argmax()), 'sagittal': int(sum_roi_sagittal.argmax()), 'coronal': int(sum_roi_coronal.argmax())}
    

In [4]:
## Create a dictionary of center slices 
def create_center_slices_dict(norm_files_dict, directions, slices, slices_above_and_below = 0):
    ## This function extracts the slices that correspond to the max value of roi
    center_slices_dict = {}
    for plane in directions: 
        slice_pick = slices[plane]
        for img in img_types: 
            center_slices_dict_key = img+"_"+plane+"_"+str(slice_pick)
            if plane == 'axial': 
                center_slices_dict[center_slices_dict_key] = norm_files_dict[img][slice_pick, :,:]
            elif plane == "sagittal": 
                center_slices_dict[center_slices_dict_key] = norm_files_dict[img][:, :,slice_pick]
            elif plane == 'coronal': 
                center_slices_dict[center_slices_dict_key] = norm_files_dict[img][:, slice_pick,:]
    
    if slices_above_and_below > 0: 
        
        for plane in directions: 
            slice_pick = slices[plane]+slices_above_and_below
            for img in img_types: 
                center_slices_dict_key = img+"_"+plane+"_"+str(slice_pick)
                if plane == 'axial': 
                    center_slices_dict[center_slices_dict_key] = norm_files_dict[img][slice_pick, :,:]
                elif plane == "sagittal": 
                    center_slices_dict[center_slices_dict_key] = norm_files_dict[img][:, :,slice_pick]
                elif plane == 'coronal': 
                    center_slices_dict[center_slices_dict_key] = norm_files_dict[img][:, slice_pick,:]
        
        for plane in directions: 
            slice_pick = slices[plane]-slices_above_and_below
            for img in img_types: 
                center_slices_dict_key = img+"_"+plane+"_"+str(slice_pick)
                if plane == 'axial': 
                    center_slices_dict[center_slices_dict_key] = norm_files_dict[img][slice_pick, :,:]
                elif plane == "sagittal": 
                    center_slices_dict[center_slices_dict_key] = norm_files_dict[img][:, :,slice_pick]
                elif plane == 'coronal': 
                    center_slices_dict[center_slices_dict_key] = norm_files_dict[img][:, slice_pick,:]
                
    return center_slices_dict

In [5]:
## CREATING ROI BOUNDS 
def roi_bounds(roi, plane, slice_pick): 
    if plane == 'axial': 
        roi_slice = roi[slice_pick, :,:]
    elif plane == 'sagittal': 
        roi_slice = roi[:, :,slice_pick]
    elif plane == 'coronal': 
        roi_slice = roi[:, slice_pick,:]
    
    if roi_slice.any(): 
        r = np.any(roi_slice, axis=(1))
        c = np.any(roi_slice, axis=(0))

        rmin, rmax = np.where(r)[0][[0, -1]]
        cmin, cmax = np.where(c)[0][[0, -1]]

        status_code = 0
        return rmin, rmax, cmin, cmax, status_code
    else: 
        rmin, rmax, cmin, cmax = None, None, None, None 
        status_code = 1
        return rmin, rmax, cmin, cmax, status_code 

In [6]:
## COMPUTING SQUARE ROI BOUNDS
def compute_square_dims_flex(roi_img, plane, slice_pick, padding = 0): 
    
    rmin, rmax, cmin, cmax, status_code = roi_bounds(roi_img, plane, slice_pick)
    
    if status_code == 0: 

        r = rmax-rmin
        c = cmax-cmin

        diff = abs(r - c)
        expand_number = int(diff/2)
    #     print(expand_number)
        if r == c: 
            print('already square.')
        if r > c: 
            if diff % 2 == 0: 
                cmax_new = cmax+expand_number+padding
                cmin_new = cmin-expand_number-padding
            else: 
                cmax_new = cmax+expand_number+1+padding
                cmin_new = cmin-expand_number-padding
            if cmin_new < 0: 
                cmin_new, cmax_new = cmin_new+abs(cmin_new), cmax_new+abs(cmin_new)
            return rmin-padding, rmax+padding, cmin_new, cmax_new, status_code

        else: 
            if diff % 2 == 0: 
                rmax_new = rmax+expand_number+padding
                rmin_new = rmin-expand_number-padding
            else: 
                rmax_new = rmax+expand_number+1+padding
                rmin_new = rmin-expand_number-padding    
            if rmin_new < 0: 
                rmin_new, rmax_new = rmin_new+abs(rmin_new), rmax_new+abs(rmin_new)

            return rmin_new, rmax_new, cmin-padding, cmax+padding, status_code 
    else: 
        return rmin, rmax, cmin, cmax, status_code 
        



In [7]:
## CREATING DICTIONARY OF CROPPED SLICES: 
def create_cropped_slices_dict(center_slices_dict, roi, padding = 0):
    cropped_slices_dict = {}
    for key, value in center_slices_dict.items(): 
#         print(key)
        img_type, plane, slice_pick = key.split('_')
        slice_pick = int(slice_pick)
        rmin, rmax, cmin, cmax, status_code = compute_square_dims_flex(roi, plane, slice_pick, padding)
        if status_code == 0: 
            if rmin < 0 or cmin < 0:
                rmin, rmax, cmin, cmax, status_code = compute_square_dims_flex(roi, plane, slice_pick, padding=0)
#             print(rmin, rmax, cmin, cmax)
            new_key = str(key)+"_dim"+str(rmax-rmin)
            cropped_slices_dict[new_key] = value[rmin:rmax, cmin:cmax]
        else: 
            continue
    return cropped_slices_dict

In [8]:
## CONCATENATING THE IMAGES + PUTTING THEM IN A DICTIONARY FOR WRITING: 
def create_imgs(cropped_slices_dict): 
    new_imgs_dict = {}
    #cropped slices dictionary 
    keys = cropped_slices_dict.keys()
    
    slices_of_interest = list(set(['_'.join((key.split('_')[-2], key.split('_')[-1])) for key in keys]))
    
    for slice_pick in slices_of_interest: 
        slices_for_img = [key for key in keys if slice_pick in key]
        slices_for_img.sort() 
        
        new_key = '_'.join((slices_for_img[0].split('_')[-3], slices_for_img[0].split('_')[-2]))
        
        adca_img = cropped_slices_dict[slices_for_img[0]]
        fla_img  = cropped_slices_dict[slices_for_img[1]]
        t1ca_img = cropped_slices_dict[slices_for_img[2]]
        
        new_img = np.dstack([t1ca_img, fla_img, adca_img])
        
        new_imgs_dict[new_key] = new_img
    
    return new_imgs_dict

## Reading in the data 

### Labels

In [9]:
labels = pd.read_csv('../datasets/tcga_subtype_data.csv')
# labels = labels. 
labels['IDH/codel subtype'].value_counts()

IDHwt               362
IDHmut-non-codel    263
IDHmut-codel        169
Name: IDH/codel subtype, dtype: int64

In [10]:
labels = labels[['track_name', 'IDH/codel subtype']]

### DTI

In [11]:
aligned_dti = pd.read_csv('aligned_dti.csv')

In [12]:
aligned_dti.head(20)

,cohort,dti,exam,patient_id,segmentation_name,adc,aligned_dti
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,11-AX_T1_POST_GD_FLAIR-81300a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...
1,TCGA-LGG-nifti,['205-dadc-52244.nii.gz'],06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,1001-T1_SE_POST-94212a_n,['205-dadc-52244.nii.gz'],['/working/lupolab/julia/tcia_analysis/dataset...
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,20-T1fl2dAx__Gd-43129a_n,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",['/working/lupolab/julia/tcia_analysis/dataset...
3,TCGA-LGG-nifti,"['601-DTI_ASSET-84919.nii.gz', '600-DTI_ASSET-...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,10-AX_T1_POST_GD_FLAIR-19430a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...
4,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '5-DTI_2.6mm...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,10-AX_T1_POST_GD_FLAIR-02819a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...
5,TCGA-LGG-nifti,"['802-DTI_2.6mm_WH-35614.nii.gz', '600-Exponen...",07-26-1997-MRI_BRAIN_WWO_CONTRAST-17402,TCGA-DU-A5TR,10-AX_T1_POST_GD_FLAIR-23755a_n,['600-Exponential_Apparent_Diffusion_Coefficie...,['/working/lupolab/julia/tcia_analysis/dataset...
6,TCGA-LGG-nifti,"['900-PROP_ADC-74032.nii.gz', '401-Apparent_Di...",10-28-1995-MRI_BRAIN_FOR_STEREOTACTIC_WWO_CONT...,TCGA-HT-7604,12-CAX_3D_SPGR-37353a_n,"['900-PROP_ADC-74032.nii.gz', '401-Apparent_Di...",['/working/lupolab/julia/tcia_analysis/dataset...
7,TCGA-LGG-nifti,"['6-DTI_2.6mm_WH-60904.nii.gz', '601-DTI_2.6mm...",11-19-1996-MRI_BRAIN_WWO_CONTRAST-27038,TCGA-DU-8163,12-AX_T1_POST_GD_FLAIR-07772a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...
8,TCGA-LGG-nifti,"['1100-DTI_ASSET-99068.nii.gz', '1101-DTI_ASSE...",12-20-1991-MRI_BRAIN_WWO_CONTRAST-98852,TCGA-DU-7018,13-AX_T1_POST_GD_FLAIR-47091a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...
9,TCGA-LGG-nifti,['4-ep2ddiff3scantraceADC-62120.nii.gz'],09-17-2001-MRI_BRAIN__W_WO_CONT-92660,TCGA-EZ-7265A,10-T1AXMPRSEL3D-83348a_n,['4-ep2ddiff3scantraceADC-62120.nii.gz'],['/working/lupolab/julia/tcia_analysis/dataset...


In [13]:
aligned_dti = aligned_dti.merge(labels, how = 'left', left_on = ['patient_id'], right_on = ['track_name'])

In [14]:
aligned_dti = aligned_dti.loc[aligned_dti.patient_id != 'TCGA-02-0087']
aligned_dti = aligned_dti.loc[aligned_dti.patient_id != 'TCGA-06-0178']
aligned_dti = aligned_dti.loc[aligned_dti.patient_id != 'TCGA-02-0075']
aligned_dti = aligned_dti.loc[aligned_dti.patient_id != 'TCGA-02-0060']
aligned_dti = aligned_dti.loc[aligned_dti.patient_id != 'TCGA-02-0086']

In [15]:
labels_dti = []
for idx, row in aligned_dti.iterrows(): 
    if row['cohort'] == 'TCGA-LGG-nifti': 
        if row['IDH/codel subtype'] == 'IDHmut-non-codel': 
            label = 'as_idh_mut'
        elif row['IDH/codel subtype'] == 'IDHwt': 
            label = 'as_idh_wt'
        elif row['IDH/codel subtype'] == 'IDHmut-codel': 
            label = 'od_1p19codel'
    else: 
        if row['IDH/codel subtype'] == 'IDHmut-non-codel': 
            label = 'gbm_idh_mut'
        elif row['IDH/codel subtype'] == 'IDHwt': 
            label = 'gbm_idh_wt'
        elif row['IDH/codel subtype'] == 'IDHmut-codel': 
            label = 'gbm_codel'
    labels_dti.append(label)

In [16]:
aligned_dti['label'] = labels_dti

In [17]:
aligned_dti.label.value_counts()

as_idh_mut      40
gbm_idh_wt      24
od_1p19codel    19
as_idh_wt       12
Name: label, dtype: int64

In [18]:
adc_dict = {row['patient_id']: row['aligned_dti'] for idx, row in aligned_dti.iterrows()}

In [19]:
os.listdir()

['dti_seg_map.csv',
 'skullstrip_lgg.txt',
 '06_map_segmented_exams_to_exams_with_DTI.ipynb',
 'register_lggs.txt_1.csh',
 '11_CreateTestImages.ipynb',
 '03_Create_Datalist.ipynb',
 'premask_condensed.txt',
 'segmentation_mapping.json',
 'skullstrip_lgg.txt_1.csh',
 'process_DTI_brain',
 '01_Register_LGG_Niftis_Using_BrainsFit.ipynb',
 'skullstrip_condensed_stripOnly.txt',
 'register_GBMs.txt_1.csh',
 '.ipynb_checkpoints',
 'src',
 '05_findDiffusionSequences.ipynb',
 'register_GBMs.txt',
 'premask_gbms.txt',
 'segmentation_mappings.json',
 '00_check_nifti_sizes.ipynb',
 '02_SkullStripCondensed.ipynb',
 '08_DTI_int2_to_nifti.ipynb',
 'final_dti_subset.csv',
 'premask_lggs.txt',
 'skullstrip_gbms.txt',
 'usable_patient_gbm.csv',
 '07_processDTI.ipynb',
 'register_ADC.txt',
 'skullstrip_condensed.txt_1.csh',
 'dti_dict.json',
 'register_ADC.txt_1.csh',
 'skullstrip_condensed.txt',
 'register_lggs.txt',
 '10_CreateRGBImages_munge.ipynb',
 'premask_gbms.txt_1.csh',
 '04_mapping_segmentation

In [20]:
aligned_dti.to_csv('aligned_dti_with_labels.csv', index = False)

### Segmentations

In [19]:
seg_dict = {row['patient_id']: row['segmentation_name'] for idx, row in aligned_dti.iterrows()}

### T1, FLAIR

Read in json file for segmentation: 

In [20]:
with open('/working/lupolab/julia/tcia_analysis/segmentation_mri_brain_tumors_br16_full_v1/config/config_datalist_stripped.json') as f: 
    datalist = json.load(f)

Creating lookup dictionaries with the modalities we care about for creating images: 

In [21]:
t1_post_dict = {x['image'][0].split('/')[1]:x['image'][0].split('/')[2]+'/'+x['image'][0].split('/')[3] for x in datalist['validation']}

In [22]:
t2_flair_dict = {x['image'][0].split('/')[1]:x['image'][3].split('/')[2]+'/'+x['image'][3].split('/')[3] for x in datalist['validation']}

## Create a main function that cycles through all possible aligned ADC images + creates RGB images 

In [23]:
file_types = ['t1ca', 'fla', 'adca', 't2all', 'cel', 'brainmask']
img_types = file_types[0:3]
roi_types = file_types[3:5]
brainmask_types = file_types[5:6] 
max_area_roi = 'cel'
directions = ['coronal', 'axial', 'sagittal']
sab = 0
crop_roi = 't2all'
crop_padding = 10
root_dir = Path('/working/lupolab/julia/tcia_analysis/datasets/')
seg_root_dir = Path('/working/lupolab/julia/tcia_analysis/segmentation_mri_brain_tumors_br16_full_v1/eval_strip/')
output_path = Path('/working/lupolab/julia/cnn_analysis/who_subtype/data/processed_images', 'crop'+crop_roi+'_maxarea'+max_area_roi+'_sab'+str(sab))

In [24]:
os.makedirs(output_path, exist_ok = True)

In [25]:
from scipy.misc import imsave

In [26]:
aligned_dti = aligned_dti.sort_index(ascending=False)

In [29]:
aligned_dti

,cohort,dti,exam,patient_id,segmentation_name,adc,aligned_dti,track_name,IDH/codel subtype,label
99,TCGA-GBM-nifti,"['9-DTIColFA-98462.nii.gz', '6-DTI-72659.nii.g...",12-10-2002-NR_MRI_BRAIN_WWO-21815,TCGA-19-2624,13-MRHR_T1_AX_POST_GAD-93013a_n,"['7-DTIADC-43987.nii.gz', '5-MRHR_DIFF_IPATADC...",['/working/lupolab/julia/tcia_analysis/dataset...,TCGA-19-2624,IDHwt,gbm_idh_wt
98,TCGA-GBM-nifti,['814-Apparent_Diffusion_Coefficien-13024.nii....,10-29-2002-MSTEALTH-52317,TCGA-08-0529,7-3D_SPGR_AX-36833a_n,['814-Apparent_Diffusion_Coefficien-13024.nii....,['/working/lupolab/julia/tcia_analysis/dataset...,NaN,NaN,gbm_idh_wt
97,TCGA-GBM-nifti,['400-Apparent_Diffusion_Coefficient_mms-23032...,10-22-2007-23597,TCGA-06-1801,8-AX_T1_FLAIR_C-45978a_n,['400-Apparent_Diffusion_Coefficient_mms-23032...,['/working/lupolab/julia/tcia_analysis/dataset...,NaN,NaN,gbm_idh_wt
96,TCGA-GBM-nifti,['5-MRHG_DIFFADC-02800.nii.gz'],10-12-2001-NR_MRI_BRAIN_WWO-55607,TCGA-19-0963,7-MRHG_T1_AX_POST_GAD-84768a_n,['5-MRHG_DIFFADC-02800.nii.gz'],['/working/lupolab/julia/tcia_analysis/dataset...,NaN,NaN,gbm_idh_wt
94,TCGA-GBM-nifti,['6-ep2ddiff3scantracep2ADC-07811.nii.gz'],02-10-2000-HeadRoutine-89271,TCGA-14-1825,10-t1tra_post-80382a_n,['6-ep2ddiff3scantracep2ADC-07811.nii.gz'],['/working/lupolab/julia/tcia_analysis/dataset...,TCGA-14-1825,IDHwt,gbm_idh_wt
93,TCGA-GBM-nifti,"['402-DTI_2.6mm_WH-04206.nii.gz', '400-DTI_2.6...",02-27-2008-MRI_BRAIN_WWO_CONTRAST-78379,TCGA-06-1806,11-AX_T1_POST_GD_FLAIR-87636a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...,TCGA-06-1806,IDHwt,gbm_idh_wt
92,TCGA-GBM-nifti,['604-dADC_MAP-57575.nii.gz'],01-22-2001-MRI_BRAIN_COMBO-10861,TCGA-76-4935,1301-T1_AX_SE_FS_POST-40917a_n,['604-dADC_MAP-57575.nii.gz'],['/working/lupolab/julia/tcia_analysis/dataset...,TCGA-76-4935,IDHwt,gbm_idh_wt
90,TCGA-GBM-nifti,['813-Apparent_Diffusion_Coefficien-54400.nii....,06-06-2001-MSTEALTH-13589,TCGA-08-0355,6-3D_SPGR_AX-29720a_n,['813-Apparent_Diffusion_Coefficien-54400.nii....,['/working/lupolab/julia/tcia_analysis/dataset...,NaN,NaN,gbm_idh_wt
89,TCGA-GBM-nifti,['600-Exponential_Apparent_Diffusion_Coefficie...,03-19-2000-MRI_BRAIN_WWO_CONTRAST-00318,TCGA-14-1454,10-Ax_T1_SE_POST-41828a_n,['600-Exponential_Apparent_Diffusion_Coefficie...,['/working/lupolab/julia/tcia_analysis/dataset...,NaN,NaN,gbm_idh_wt
88,TCGA-GBM-nifti,['6-DIFFUSIONADC-19007.nii.gz'],03-15-2004-NR_MRI_BRAIN_WWO-30388,TCGA-19-5960,9-T1_AXIAL_Gd-31768a_n,['6-DIFFUSIONADC-19007.nii.gz'],['/working/lupolab/julia/tcia_analysis/dataset...,TCGA-19-5960,IDHwt,gbm_idh_wt


In [30]:
file_loc_and_label = pd.DataFrame()

for idx, row in aligned_dti.iterrows(): 
    
    # Set patient ID, cohort 
    patient_id     = row['patient_id']
    print(patient_id)
    cohort         = row['cohort']
    brainmask_name = '.'.join(t1_post_dict[patient_id].split('.')[:-2])+'_mask.nii.gz'
    if '[' in adc_dict[patient_id]: 
        adc_name   = '/'.join(ast.literal_eval(adc_dict[patient_id])[0].split('/')[-2:])
    else: 
        adc_name   = adc_dict[patient_id]
    label = row['label']
    
    # Read in volumes: 
    t1c_vol       = nib.load(Path(root_dir, cohort, patient_id, t1_post_dict[patient_id]))
    fla_vol       = nib.load(Path(root_dir, cohort, patient_id, t2_flair_dict[patient_id]))
    adc_vol       = nib.load(Path(root_dir, cohort, patient_id, adc_name))
    brainmask_vol = nib.load(Path(root_dir, cohort, patient_id, brainmask_name))
    t2all_vol     = nib.load(Path(seg_root_dir, seg_dict[patient_id], seg_dict[patient_id]+'_WT.nii.gz'))
    cel_vol       = nib.load(Path(seg_root_dir, seg_dict[patient_id], seg_dict[patient_id]+'_WT.nii.gz'))

    # Create the files dictionary:
    files_dict = {'t1ca': np.array(t1c_vol.dataobj), 
                  'fla': np.array(fla_vol.dataobj), 
                  'adca': np.array(adc_vol.dataobj), 
                  'cel': np.array(cel_vol.dataobj), 
                  't2all': np.array(t2all_vol.dataobj), 
                  'brainmask': np.array(brainmask_vol.dataobj)}

    # Check for empty ROIs 
    if files_dict['t2all'] is not None: 
        if files_dict['t2all'].any(): 
            status_code = 0
        else: 
            if files_dict['cel'].any(): 
                status_code = 0
            else: 
                status_code = 1
                print('Patient ID = %s has an empty ROIs.' %patient_id)
                break
    
    # If both ROIs are not empty: 
    if status_code == 0: 
        
        # NORMALIZE: 
        norm_files_dict = {}
        for file_type in img_types: 
            if files_dict['brainmask'] is not None: 
                norm_files_dict[file_type] = zscore_normalize(files_dict[file_type], files_dict['brainmask'])
            else: 
                print('Empty brainmask for %s' %patient_id)
        for file_type in roi_types: 
            norm_files_dict[file_type] = files_dict[file_type]
        for file_type in brainmask_types: 
            norm_files_dict[file_type] = files_dict[file_type]
            
        
        # FIND SLICES OF INTEREST: 
        ## defining the "other" roi in case max_area_roi is empty. 
        other_roi = [x for x in roi_types if x != max_area_roi][0]
        if norm_files_dict[max_area_roi] is not None: 
            if norm_files_dict[max_area_roi].any(): 
                slices = find_slices_with_max_area_roi(norm_files_dict[max_area_roi])
            else: slices = find_slices_with_max_area_roi(norm_files_dict[other_roi])
        else: slices = find_slices_with_max_area_roi(norm_files_dict[other_roi])
            
        # CREATE DICTIONARY OF SLICES OF INTEREST: 
        center_slices_dict = create_center_slices_dict(norm_files_dict, directions, slices, slices_above_and_below=sab)
        
        # CREATE DICTIONARY OF CROPPED CENTER SLICES BASED ON CROP ROI 
        other_other_roi = [x for x in roi_types if x != crop_roi][0]
        
        if norm_files_dict[crop_roi] is not None: 
            if norm_files_dict[crop_roi].any(): 
                cropped_slices_dict = create_cropped_slices_dict(center_slices_dict, norm_files_dict[crop_roi], padding = crop_padding)
            else: print('no %s' %crop_roi)
        elif norm_files_dict[other_other_roi] is not None: 
            if norm_files_dict[other_other_roi].any(): 
                cropped_slices_dict = create_cropped_slices_dict(center_slices_dict, norm_files_dict[other_other_roi], padding = crop_padding)
            else: print('no %s' %crop_padding)
        else: 
            print("No ROI, skipping.")
            status_code = 1          
        
        # CYCLE THROUGH CROPPED SLICES DICT & CREATE AN IMAGE FOR EACH: 
        if status_code == 0: 
            new_imgs_dict = create_imgs(cropped_slices_dict)
            for key in new_imgs_dict.keys(): 
                img_name = str(patient_id+"_"+key+'.png')
                imsave(Path(output_path/img_name), new_imgs_dict[key])
                file_loc_and_label_line = {'location':output_path, 'file_name': img_name, 'label': label}
                file_loc_and_label = file_loc_and_label.append(file_loc_and_label_line, ignore_index = True)
        

TCGA-19-2624


/data/svcf/software/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:92: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


TCGA-08-0529
TCGA-06-1801
TCGA-19-0963
TCGA-14-1825
TCGA-06-1806
TCGA-76-4935
TCGA-08-0355
TCGA-14-1454
TCGA-19-5960
TCGA-12-1098
TCGA-19-0955


KeyboardInterrupt: 

In [ ]:
file_loc_and_label.to_csv(Path(output_path, 'labels.csv'))

In [ ]:
file_loc_and_label.location[0]